In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import DataSet 'Data Analyst Jobs'

In [ ]:
# IMPORT DATA AND PACKED
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots 
from plotly.offline import init_notebook_mode, iplot

In [ ]:
# Conditions all for the plotting
sns.set()
pd.options.plotting.backend = 'plotly'
plt.style.use('seaborn')
init_notebook_mode(connected = True)

In [ ]:
#Charge data csv 'data-analyst-jobs' as format Dataframe from pandas
df = pd.read_csv('../input/data-analyst-jobs/DataAnalyst.csv')
df.head()

In [ ]:
# General info from data:
df.info()

# Data Cleaning to be Worked On

In [ ]:
# Delete column "Unnamed: 0"
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
# Copy data a new DataFrame
df1 = df.copy()
df1.head(2)

In [ ]:
# DATA CLEANING

#Separate values for create new two fields from them:
df1['Job Title'],df1['Department']= df['Job Title'].str.split(',',1).str
df1['Company Name'],_ = df['Company Name'].str.split('\n',1).str
df1['Salary Estimate'],_= df['Salary Estimate'].str.split('(',1).str
df1['Min Salary'],df1['Max Salary']= df1['Salary Estimate'].str.split('-').str

# Created values Max Salary and Min Salary as well as data cleaning with strip methods from string class
# Replace withe spaces for nan values through the numpy.
clean_salary = lambda x: x.replace('', np.nan).str.strip().str.lstrip('$').str.rstrip('K').fillna(0).astype(int)
df1['Min Salary'] = clean_salary(df1['Min Salary'])
df1['Max Salary'] = clean_salary(df1['Max Salary'])

# Empty Field 'Salary Estimate' but created Max and Min salary.
df1 = df1.drop(['Salary Estimate'],axis = 1)

# Show 'df1' with the changes realized previously. 
df1.head()

# Companies Easy Apply for Data Analyst Jobs

In [ ]:
# DATA ANALYTICS
# Count values from easy apply for each job offers
df1['Easy Apply'].value_counts()

In [ ]:
# Easy applicated only 80 companies, the rest could to be applicated dificult it.
# With a function, it recodes key and values to take new values between 0 and 1
def recode(column, new_code):
    col_cod = pd.Series(column, copy = True)
    for key, values in new_code.items():
        col_cod.replace(key, values, inplace = True)
    return col_cod

In [ ]:
# Use the new function
df1['Easy Apply'] = recode(df1['Easy Apply'],{'-1':0,'True':1})
df1['Competitors'] = recode(df1['Competitors'],{'-1':np.nan})

# Extracted data the Easy Apply with values equal '1', in the other words,  
# this part will to make only in offers jobs where is easy applicated
df_easy_apply = df1[df1['Easy Apply'] == 1]
df_easy_apply.reset_index()
df_easy_apply.head()

In [ ]:
# New Dataframe through method 'Groupby' from 'pandas' packet. It's grouped in fallen order and count the values "Easy Apply" for each offer jobs
df_easy_apply_1 = df_easy_apply.groupby(['Company Name','Sector'])['Easy Apply'].count().reset_index()
# In the same way, apply method 'Groupby' with the values "Max Salary". It's grouped throght it's average
df_easy_apply_2 = df_easy_apply.groupby(['Company Name'])['Max Salary'].mean().reset_index()
# Apply concate fields and order descendent for dataframe before and perform cut in ten primary
df_easy = pd.DataFrame()
df_easy = pd.concat([df_easy_apply_1['Company Name'], df_easy_apply_1['Easy Apply'], 
                     df_easy_apply_2['Max Salary']], axis = 1)
df_easy = df_easy.sort_values('Easy Apply', ascending = False).head(10).reset_index().drop('index', axis = 1)
df_easy.head(10)

In [ ]:
# Plot graph bar with Plotly included color graduation in function the Max Salary Mean.
chart = px.bar(df_easy, y = 'Easy Apply', x = 'Company Name', color = 'Max Salary',
              color_continuous_scale = px.colors.sequential.Viridis,
              title = "Companies That Have Jobs as Data Analyst With Easily Apply")
chart.show()

# **Size Company - Employed Quantity that Offer Jobs as a Data Analyst**

In [ ]:
# Now, visualization of size company in funtion quantily employees that offers job in data analyst.

# Create DateFrame and empty rows with useless data
# Change of field names for easy descriptive in graphic and visualization
df_employed = df1['Size'].value_counts().to_frame().reset_index()
df_employed = df_employed.drop(df_employed[(df_employed['index'] == '-1') | \
                                           (df_employed['index'] == 'Unknown')].index, 
                               axis = 0).rename(columns = {'index': 'Size Company',
                                                           'Size': 'Nº Company'})
df_employed.head()

In [ ]:
# Finally, plot graphic barplot in horizontal for Size Company in function of employees Quantity:
# Use Plotly Express
chart1 = px.bar(df_employed, x = 'Nº Company', y = 'Size Company',
               title = 'Size Company in function of employees Quantity')
chart1.show()

# Classification of Companies vs Ratings

In [ ]:
# Use grouby method for get a new dataframe thad measure number companies your rating
# In addition, we order descending in the ratings function, discard lower rating values that 3.00 
# And rename fields
df_rating = df1.groupby('Rating')['Company Name'].count().to_frame().reset_index()\
                        .sort_values('Rating', ascending = False).reset_index()\
                        .drop('index', axis = 1)\
                        .rename(columns = {'Company Name': 'Nº Companies'})

df_rating = df_rating.drop(df_rating[(df_rating['Rating'] <= 3.00)].index, axis = 0)

df_rating.head()

In [ ]:
# Plot visualization in graphic bars. 
plt.figure(figsize = (12,6))

plot = sns.barplot(x = 'Rating', y = 'Nº Companies', data = df_rating, palette = 'rocket')
plot = plot.set_xticklabels(plot.get_xticklabels(),
                           horizontalalignment = 'right',
                           fontweight = 'light')

# Ownership Companies Type that Offers Jobs Data Analyst

In [ ]:
# Visualization of Ownerships companies type that offers job Data Analysta
# Empty row with '-1 index, rename fields and discard lowers values that 10
df_ownership = df1['Type of ownership'].value_counts().to_frame()\
               .drop('-1').reset_index()\
               .rename(columns = {'index': 'Type of Ownership',
                                  'Type of ownership': 'Nº Offers Jobs'})

df_ownership = df_ownership.drop(df_ownership[(df_ownership['Nº Offers Jobs'] <= 10)].index, axis = 0)
df_ownership.head()

In [ ]:
# Finaly, barplot with seaborn
plt.figure(figsize = (12,6))
plot = sns.barplot(x = 'Nº Offers Jobs', y = 'Type of Ownership', data = df_ownership)
plot = plot.set_xticklabels(plot.get_xticklabels(),
                           rotation = 65,
                           horizontalalignment = 'right')

# Range Salary for Each Sector

In [ ]:
# Now, analyze Max salary in fuction the each sector
# Empty rows and fields innesesary
df_mean_salary = df1.groupby(['Sector']).mean().reset_index()\
                    .drop(['Founded','Easy Apply'], axis = 1)
# The sectors as government and non-profit it would't take for analitycs
df_mean_salary = df_mean_salary.drop(df_mean_salary[(df_mean_salary['Sector'] == '-1') | \
                                                    (df_mean_salary['Sector'] == 'Government') | \
                                                    (df_mean_salary['Sector'] == 'Non-Profit')].index,
                                     axis = 0) \
                                     .sort_values('Max Salary', ascending = False)

df_mean_salary.head()

In [ ]:
# Visualization in bar graph with plotly

# Create Trace 1
trace1 = go.Bar(x = df_mean_salary['Sector'],
                y = df_mean_salary['Max Salary'],
                name = 'Max Salary',
                marker = dict(color ='rgb(55, 83, 109)',
                             line = dict(color = 'rgb(0,0,0)', width = 1.5)))

# Create Trace 2
trace2 = go.Bar(x = df_mean_salary['Sector'],
                y = df_mean_salary['Min Salary'],
                name = 'Min Salary',
                marker = dict(color = 'indianred',
                             line = dict(color = 'rgb(0,0,0)', width = 1.5)))

data = [trace1, trace2]

layout = go.Layout(title = 'Salary Range For Each Sector', 
                   barmode = 'group')

chart = go.Figure(data = data, layout = layout)
iplot(chart)


# Distribution of Job Offers as a Data Analyst in the Territory of the United States

In [ ]:
# Create a DataFrame with features (Mean Salary, Location and Mean Salary) 
df_states = pd.DataFrame()
dftemp = df1.copy()
dftemp['Mean Salary'] = (df1['Max Salary'] + df1['Min Salary']) / 2
df_states = pd.concat([df1['Sector'],df1['Location'], dftemp['Mean Salary']], axis = 1)
df_states.head()

In [ ]:
# Seperate Acronym and City from Location Field
df_states['City'], df_states['Acronym State'] = df_states['Location'].str.split(',', 1).str
# Remove field "Location" and values '-1' in field sector:
df_states = df_states.drop('Location', axis = 1) \
                     .drop(df_states[df_states['Sector'] == '-1'].index, axis = 0)
df_states.head()

In [ ]:
# Apply Groupy with DataFrame both in the count and the mean values, after concated fields in the new DataFrame:
df_states_group = df_states.groupby(['Acronym State']).count().reset_index()
df_salary_group = df_states.groupby(['Acronym State']).mean().reset_index()

# New Dataframe 
df_map = pd.DataFrame()
df_map = pd.concat([df_states_group['Acronym State'], 
                    df_states_group['Sector'], 
                    df_salary_group['Mean Salary']], 
                   axis = 1)
df_map.head()

In [ ]:
# Round Mean Salary Value with Lambda function and remove row in 'Araphone, CO' Value of "Sector" field
df_map = df_map.drop([1], axis = 0).rename(columns = {'Sector': 'Nº Offers Jobs'})
df_map['Mean Salary'] = df_map['Mean Salary'].apply(lambda x: round(x, 2))
df_map.head()

In [ ]:
# Plot Map of United States with Plotly in module graph_objects:
locations = list(df_map['Acronym State'])

# Remove withe space in "Acronym State" field
for i in range(0, len(locations)):
    locations[i] = locations[i].replace(" ", "")

qjobs = list(df_map['Nº Offers Jobs'])

# Create text that displays the basic data when the cursor is over the chart status 
df_map_str = df_map.copy()
for col in df_map_str.columns:
    df_map_str[col] = df_map_str[col].astype(str)
    
# Apply '$' symbol in Mean Salary values
df_map_str['Mean Salary'] = df_map_str['Mean Salary'].apply(lambda x: "$" + x + "/Year")
# Text visualization:
df_map_str['Text'] = df_map_str['Acronym State']  + '<br>' + \
    "Nª Offers Jobs: " + df_map_str['Nº Offers Jobs'] + '<br>' \
        "Mean Salary from the Jobs as Data Analyst: " + '<br>' + df_map_str['Mean Salary'] + " (Thousands)"
df_map_str.head()

In [ ]:
# Plot Map in plotly
import plotly.graph_objects as go


fig = go.Figure(data = go.Choropleth(locations = locations,
                                    z = qjobs,
                                    locationmode = 'USA-states',
                                    text = df_map_str['Text'],
                                    marker_line_color = 'black',
                                    colorbar_title = 'Nº Offers Jobs'),) 

fig.update_layout(title_text = 'Nº Offers Jobs as Data Analyst<br>United States of America',
                 geo = dict(scope = 'usa',
                           projection = go.layout.geo.Projection(type = 'albers usa'),
                           showlakes = True,
                           lakecolor = 'rgb(255,255,255)'),)
fig.show()

In [ ]:
# Created new dataframe for view jobs percentage as well as its mean salary year for each state
df_map2 = df_map.copy()
df_map2['Jobs Percentage'] = (df_map2['Nº Offers Jobs'] / df_map2['Nº Offers Jobs'].sum()) * 100
df_map2['Jobs Percentage'] = df_map2['Jobs Percentage'].apply(lambda x: round(x, 1))
df_map2 = df_map2.sort_values('Jobs Percentage', ascending = True) \
                 .drop(df_map2[df_map2['Jobs Percentage'] <= 2.0].index, axis = 0) \
                 .reset_index() \
                 .drop('index', axis = 1)
df_map2.head(11)

In [ ]:
# Create two Subplots:
fig_states = make_subplots(rows = 1, cols = 2, specs = [[{},{}]], shared_xaxes = True,
                          shared_yaxes = False, vertical_spacing = 0.001)

fig_states.append_trace(go.Bar(x = df_map2['Jobs Percentage'],
                       y = df_map2['Acronym State'],
                       marker = dict(color = 'rgba(50, 171, 40, 0.7)',
                                    line = dict(color='rgba(60, 171, 120, 1.0)',
                                               width = 1,)
                                    ),
                       name = 'Percentage Jobs',
                       orientation = 'h',
                       ),
                1, 1)

fig_states.append_trace(go.Scatter(x = df_map2['Mean Salary'],
                                  y = df_map2['Acronym State'],
                                  mode = 'lines+markers',
                                  line_color = 'rgb(128, 0, 128)',
                                  name = 'Mean Salary'),
                       1, 2)

fig_states.update_layout(title = 'Distribution of the Offers Jobs From in United States',
                        yaxis = dict(showgrid = True,
                                    showline = True,
                                    showticklabels = True,
                                    linecolor = 'rgba(102,102,102,0.8)',
                                    linewidth = 2,
                                    domain = [0, 0.94],
                                    ),
                        yaxis2 = dict(showgrid = True,
                                     showline = True,
                                     showticklabels = False,
                                     linecolor = 'rgba(102,102,102,0.8)',
                                     linewidth = 1,
                                     domain = [0, 0.94],
                                     ),
                        xaxis = dict(zeroline = False,
                                    showline = False,
                                    showticklabels = True,
                                    showgrid = True,
                                    domain = [0, 0.40],
                                    dtick = 5,
                                    ),
                        xaxis2 = dict(zeroline = False,
                                     showline = False,
                                     showticklabels = True,
                                     showgrid = True,
                                     domain = [0.47, 1],
                                     side = 'top',
                                     dtick =8,
                                     ),
                        legend = dict(x = 0.02, y = 1.07, font_size = 12),
                        margin = dict(l = 100, r = 20, t = 70, b = 70),
                        paper_bgcolor='rgb(250, 248, 255)',
                        plot_bgcolor='rgb(250, 248, 255)',)

fig_states.show()

# Hard Skills That it is Most Requested per The Companies

In [ ]:
# A dictionary is created with a list of hard skills to search in the job description fields
import re #Library for regular expresions 

hard_skills_dict = {
    'Python': r"python",
    'R': r"[\b\s/]r[\s,\.]",
    'Excel': r"excel",
    'SQL': r"sql",
    'NoSQL': r"\bNo[\s,-]sql[\s]",
    'PowerBI': r"power[\s]BI",
    'Tableau': r"tableau",
    'SPSS': r'\bSPSS\b',
    'Big Data': r"\sbig\sdata\s",
    'SAP BI': r"SAP[\s]BI",
    'MongoDB': r"MongoDB",
    'Hadoop': r"Hadoop",
    'SAS': r"\bSAS\b",
    'VBA': r"\bvba\b",
    'AWS': r"\baws\b",
    'Git': r"\bGit",
    'QlikView': r"\bQlikView",
    'Oracle BI': r"oracle[\n]BI",
    'Scala': r"Scala",
    'Dashboard': r"\bDashboard[s]",
    'Spark': r"Spark",
    'Matlab': r"Matplotlib",
    'Linux': r"linux",
    'Unix': r"unix",
    'Looker': r"looker",
    'C# or C++': r"\bC[#\+\+]",
    'Java': r"java",
    'PowerPivot': r"Power[\s]Pivot",
    'PowerQuery': r"Power[\s]Query",
    'BigQuery': r"Big[\s]Query",
    'Apache Cassandra': r"[\b\s]Cassandra[\b\s]",
    'Neo4j': r"Neo4j",
    'TensorFlow': r"TensorFlow"
}

hard_skills_count = {}

# Loop through skills for count the frecuency in Jobs Description Field.
for key, search in hard_skills_dict.items():
    hard_skills_count[key] = df1['Job Description'].str.contains(search, flags = re.IGNORECASE).sum()

df_skills = pd.DataFrame.from_dict(hard_skills_count, orient = 'index') \
                        .sort_values(0, ascending = False) \
                        .reset_index() \
                        .rename(columns = {'index': 'Skills', 0: 'Count'})

df_skills['Relative Frecuency'] = (df_skills['Count'] / sum(df_skills['Count'])) *100
df_skills['Relative Frecuency'] = df_skills['Relative Frecuency'].apply(lambda x: round(x, 2))

# Remove values less than '1' per cent in its Relative Frequency
df_skills = df_skills.drop(df_skills[df_skills['Relative Frecuency'] < 1.00].index, axis = 0)

df_skills.head(df_skills.shape[0])

In [ ]:
# Plot visualization the tops hards skills.
chart_skills = px.bar(df_skills, x = 'Skills', y = 'Count',
                     color = 'Relative Frecuency',
                     labels = {'Skills': 'Hard Skills','Count': 'Nº of Requests'})
chart_skills.show()

# Cluster Analysis (K-Means) and PCA

In [ ]:
# A cluster study is performed in a data frame for the data in order to find similarities
# In addition, this is supported with PCA study
# Import Library
from sklearn.decomposition import PCA # For PCA dimensionality reduction
from sklearn.cluster import KMeans    # Method for Cluster Kmeans
from sklearn.preprocessing import QuantileTransformer #Realize Scaler Data
# For visualization from 3D plot
import plotly.express as px

In [ ]:
# Create Data Frame with "Mean Salary, Founded Years and Rating" for cluster method and PCA
df2 = df1.copy()
df2 = df2.drop(df2[df2['Founded'] == -1].index, axis = 0)
df2 = df2.drop(df2[df2['Rating'] == -1.0].index, axis = 0)
df2['Years Founded'] = (2020 - df2['Founded'])
df2['Mean Salary'] = (df2['Max Salary'] + df2['Min Salary']) / 2

# Create DataFrame
df3 = pd.DataFrame()
df3 = pd.concat([df2['Rating'], df2['Mean Salary'], df2['Years Founded']], axis = 1)
df3.head()

In [ ]:
# Rescale values with QuantileTransformer class
df3_scaler = QuantileTransformer().fit_transform(df3)
print(df3_scaler)

In [ ]:
# PCA calculation is performed with a reduction of dimensionality equal to 2
pca = PCA(n_components = 2)
df_component = pca.fit_transform(df3_scaler)
print(df_component.shape, df3_scaler.shape)

In [ ]:
# The clusters numbers is determined to minimize inertian infracluster through elbow method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', n_init = 10, 
                   max_iter = 300, random_state = 0)
    kmeans.fit(df3_scaler)
    wcss.append(kmeans.inertia_)

In [ ]:
#Plot curve inertian infracluster
fig = plt.figure(figsize = (8,7))
plt.plot(range(1,11), wcss)
plt.scatter(4, wcss[3], c = 'red',s = 200)
plt.text(4 + 0.4, wcss[3], s = '4 - Clusters', fontsize = 14)
plt.xlabel('Nº Clusters', fontsize = 14)
plt.ylabel('Inertian Infraclusters', fontsize = 14)
plt.title('Elbow Method', fontsize = 14)

In [ ]:
# 4 clusters are selected, at that elbow point an optimized cluster can be achieved.
kmeans = KMeans(n_clusters = 4, init = 'k-means++', n_init = 10, 
                max_iter = 300, random_state = 0)
kmeans.fit(df3_scaler)
y_predict = kmeans.predict(df3_scaler)
print(y_predict)

In [ ]:
# First one, 3D visualization are better for show clusters calculated
colors = ['crimson','yellow', 'indigo', 'lightseagreen']

fig3d = px.scatter_3d(df3, x = 'Rating', y = 'Mean Salary', z = 'Years Founded', 
                      color = recode(pd.Series(pd.Categorical(y_predict, [0,1,2,3,4])),
                                    {0: 'Cluster-1',
                                     1: 'Cluster-2',
                                     2: 'Cluster-3',
                                     3: 'Cluster-4'}),
                      color_discrete_sequence= colors,
                      opacity = 0.9, 
                      size_max = 5)

fig3d.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig3d.show()

In [ ]:
# Graph PCA component with cluster predict
dfsns = pd.DataFrame(df_component)
# Rename Columns:
dfsns = dfsns.rename(columns = {0: 'Component 1', 1: 'Component 2'})

#Plot with Seaborn
sns.set_style('darkgrid')
sns.relplot(x = 'Component 1', 
            y = 'Component 2', 
            hue = recode(pd.Series(pd.Categorical(y_predict, [0,1,2,3,4])),
                        {0: 'Cluster-1',
                         1: 'Cluster-2',
                         2: 'Cluster-3',
                         3: 'Cluster-4'}), 
            data = dfsns, 
            palette= colors)

In [ ]:
df_predict = pd.DataFrame(y_predict)
df_predict = df_predict.rename(columns = {0 : 'Cluster'})
df_predict.head()

In [ ]:
# Dataframe with cluster classification
df3_scaler = pd.DataFrame(df3_scaler)
df3_scaler = df3_scaler.rename(columns = {0: 'Rating', 1: 'Mean Salary', 2: 'Years Founded'})
df3_scaler['Cluster'] = df_predict['Cluster']
df3_scaler['Cluster'] = recode(df3_scaler['Cluster'], 
                               {0: 'Cluster-1', 
                                1: 'Cluster-2', 
                                2: 'Cluster-3',
                                3: 'Cluster-4'}
                              )
df3_scaler.head()

In [ ]:
# Groupby apply
df_cluster_group = df3_scaler.groupby('Cluster').mean()
df_cluster_group.head(df_cluster_group.shape[0])

In [ ]:
# RADAR PLOT:
colors = ['crimson','gold', 'darkcyan', 'magenta']
y = np.array(df_cluster_group)
categories = list(df_cluster_group.columns)
radar_chart = go.Figure()

#Cluster 1
radar_chart.add_trace(go.Scatterpolar(r = y[:1,:].tolist()[0],
                                     theta = categories,
                                     fill = 'toself',
                                     name = 'Cluster-1',
                                     marker = dict(color = colors[0])
                                     ))

#Cluster 2
radar_chart.add_trace(go.Scatterpolar(r = y[1:2,:].tolist()[0],
                                     theta = categories,
                                     fill = 'toself',
                                     name = 'Cluster-2',
                                     marker = dict(color = colors[1])
                                     ))

#Cluster 3
radar_chart.add_trace(go.Scatterpolar(r = y[2:3,:].tolist()[0],
                                     theta = categories,
                                     fill = 'toself',
                                     name = 'Cluster-3',
                                     marker = dict(color = colors[2])
                                     ))

#Cluster 4
radar_chart.add_trace(go.Scatterpolar(r = y[3:4,:].tolist()[0],
                                     theta = categories,
                                     fill = 'toself',
                                     name = 'Cluster-4',
                                     marker = dict(color = colors[3])
                                     ))

# Update Layout
radar_chart.update_layout(polar = dict(radialaxis = dict(visible = True,
                                                        range = [0.0, 0.8])),
                         showlegend = True,
                         title = "Graph Radar About Principal Fields")


radar_chart.show()

In [ ]:
# Plot clusters in graph bars to identify to better
fig_bars = make_subplots(rows = 1, cols = 4, shared_yaxes = True)


# Cluster 1
fig_bars.add_trace(go.Bar(x = categories, 
                          y = y[:1,:].tolist()[0],
                          marker = dict(color = colors,),
                          name = 'Cluster - 1'),
                   row = 1, 
                   col = 1)
# Cluster 2
fig_bars.add_trace(go.Bar(x = categories, 
                          y = y[1:2,:].tolist()[0],
                          marker = dict(color = colors),
                          name = 'Cluster - 2'),
                   row = 1,
                   col = 2)

# Cluster 3
fig_bars.add_trace(go.Bar(x = categories, 
                          y = y[2:3,:].tolist()[0],
                          marker = dict(color = colors), 
                          name = 'Cluster - 3'),
                   row = 1,
                   col = 3)

# Cluster 4
fig_bars.add_trace(go.Bar(x = categories, 
                          y = y[3:4,:].tolist()[0],
                          marker = dict(color = colors), 
                          name = 'Cluster - 4'),
                   row = 1,
                   col = 4)

fig_bars.update_layout(showlegend = False, title = 'Cluster Behavior')

fig_bars.show()